In [1]:
import unibox as ub

df = ub.loads("hf://incantor/desu-0-150k-dataset-3subsets_weighted").to_pandas()

df.head()

2025-12-30 22:52:44 [INFO] loads: Loading from hf://incantor/desu-0-150k-dataset-3subsets_weighted


,image_path,caption,captioned_with,character_tags,rating,image_width,image_height,tagger_rating,tagger_character,tagger_general,subset_from,predicted_label_3cat,predicted_label_3cat_confidence,repeat_count
0,/data/yada/data/qfty_celery/dit02-desu_refined...,A young male character with multicolored blue ...,captioner_146,"aoyagi touya, shinonome akito, monokuma",general,1248,1824,None,None,None,None,star_4,0.523725,2
1,/data/yada/data/qfty_celery/dit02-desu_refined...,"hoshikawa sara, soles, feet, no shoes, heteroc...",tagger_pixai,hoshikawa sara,questionable,1248,1824,None,None,None,None,star_5,0.472817,4
2,/data/yada/data/qfty_celery/dit02-desu_refined...,A young woman with blonde hair styled in a hig...,captioner_146,"mordred (fate), mordred (swimsuit rider) (fate...",sensitive,1536,1536,None,None,None,None,star_4,0.497459,2
3,/data/yada/data/qfty_celery/dit02-desu_refined...,"hinoshita kaho, watson amelia, 1girl, glasses,...",tagger_pixai,"hinoshita kaho, watson amelia",general,1248,1824,None,None,None,None,star_4,0.733662,2
4,/data/yada/data/qfty_celery/dit02-desu_refined...,"ui (blue archive), ui (swimsuit) (blue archive...",tagger_pixai,"ui (blue archive), ui (swimsuit) (blue archive)",sensitive,1248,1824,None,None,None,None,star_4,0.645665,2


In [2]:
import lenslet

datasets = {
    "dsu-150k": df['image_path'].tolist(),
}

# Launch in non-blocking mode (returns immediately)
lenslet.launch(datasets, blocking=False, port=7070)


┌─────────────────────────────────────────────────┐
│                   🔍 Lenslet                    │
│         Lightweight Image Gallery Server        │
├─────────────────────────────────────────────────┤
│  Datasets:  dsu-150k                            │
│  Images:    227616                              │
│  Server:    http://127.0.0.1:7070                     │
│  Mode:      Subprocess (non-blocking)           │
│  PID:       141919                              │
└─────────────────────────────────────────────────┘

Gallery running at: http://127.0.0.1:7070



In [3]:
len(df['image_path'].tolist())

227616

In [ ]:
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor, as_completed
import shutil
import os


def fast_copy_files(paths, target_dir, workers=None):
    """
    Copy many files into a single directory efficiently.

    Parameters
    ----------
    paths : list[str | Path]
        List of file paths to copy.
    target_dir : str | Path
        Destination directory.
    workers : int | None
        Number of worker threads. Default uses min(32, cpu_count*2).
    """

    target_dir = Path(target_dir)
    target_dir.mkdir(parents=True, exist_ok=True)

    # Choose a good default worker count for I/O bound work
    if workers is None:
        workers = min(32, (os.cpu_count() or 4) * 2)

    def _copy_one(src):
        src = Path(src)
        dst = target_dir / src.name
        shutil.copy2(src, dst)   # Uses kernel copy when possible
        return src

    with ThreadPoolExecutor(max_workers=workers) as pool:
        futures = [pool.submit(_copy_one, p) for p in paths]

        # Force completion & surface exceptions early
        for f in as_completed(futures):
            f.result()


fast_copy_files(df['image_path'].tolist(), "/data/yada/datasets/qft/desu-0-150k-dataset-3subsets_weighted_copy")